In [1]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import time
from threading import Thread
from threading import Event
from SpArchClasses import MatrixAFetcher
from SpArchClasses import DistanceListBuilder
from SpArchClasses import MatrixBPrefetcher
from SpArchClasses import MultiplierArray
from SpArchClasses import MergeTree
from SpArchClasses import Scheduler

In [2]:

gen = np.random.default_rng()
data1 = gen.integers(1,100,10000)
row1 = gen.integers(0,100,10000)
col1 = gen.integers(0,100,10000)

data2 = gen.integers(1,100,10000)
row2 = gen.integers(0,100,10000)
col2 = gen.integers(0,100,10000)
i1 = csr_matrix(coo_matrix((data1, (row1, col1)), shape=(100, 100)).toarray())
i2 = csr_matrix(coo_matrix((data2, (row2, col2)), shape=(100, 100)).toarray())

In [3]:
print(i1.toarray())

[[  0   0  62 ...  99  69  85]
 [  0   0 107 ...   0  62  45]
 [  0   0 101 ...  48 110   0]
 ...
 [ 73   0   0 ...   0   0  38]
 [201   2   0 ...  77  71 115]
 [ 59  86   0 ...   0   0   0]]


In [4]:
print(i2.toarray())

[[ 92   0   0 ...   0   8   0]
 [ 96  64  40 ... 153   0   0]
 [  0   0   1 ...  83  58 137]
 ...
 [  0  93   0 ... 167   0 158]
 [  0  98  32 ...  87  47  18]
 [ 53  73   0 ...   0  87   0]]


In [5]:
AL = MatrixAFetcher(i1)
DL = DistanceListBuilder()
MBP = MatrixBPrefetcher(i2)
MPA = MultiplierArray()
MT = MergeTree()
S = Scheduler(AL,MT,DL,MBP)

AL.setDLB(DL)

DL.setMBP(MBP)
DL.setMultiplier(MPA)

MBP.setMultiplier(MPA)
MBP.setDistanceListBuilder(DL)

MPA.setMergeTree(MT)
MPA.setScheduler(S)

MT.setScheduler(S)

In [6]:
S.schedule()

[]
[]


In [27]:
while AL.inputFlag:
    AL.cycle()
print(AL.currentCondensedColumn)
print(AL.currentNumColumns)
print(AL.inputFlag)

0
63
False


In [8]:
AL.inputFlag

False

In [9]:
#DL.lookaheadFIFO

In [10]:
print(MBP.inputFlag)

True


In [11]:
print(MT.merging)

False


In [28]:
while not MT.merging:
    MBP.cycle()
    MPA.cycle()

    MPA.loadInputs()


In [29]:
for x in MT.inputFifos:
    print(len(x[0]))

6472
6465
6223
6207
6327
6421
6355
6444
6287
6181
6152
6192
6357
6323
6309
6396
6220
6200
6158
6123
6159
6084
6249
6258
6317
6374
6399
6447
6501
6463
6490
6356
6315
6259
6285
6216
6214
6304
6332
6286
6340
6239
6246
6292
6272
6335
6319
6321
6357
6394
6327
6382
6324
6169
6120
5982
5596
5417
4886
4691
4380
3635
3283
3926


In [30]:
print(MT.merging)
while MT.merging:
    MT.cycle()
    #for x in MT.fifos:
    #    print(x)
    #for i,fifo in enumerate(MT.fifos):
    #    print(i, fifo)
    #print(MT.out)


True
[]
[1]


In [15]:
#print(MT.fifos)

In [16]:
#len(MT.fifos[-1][0][0])

In [17]:
#print(MBP.prefetchBufferHash)

In [18]:
#print(MBP.furthestRowInPrefetcher)

In [19]:
#print(MPA.Temp_Brow)
#print(MPA.Temp_Bcols)
#print(MPA.Temp_Avalue)

In [20]:
#print(MT.inputFifos)
#print(MT.inputFifos[0])
#print(len(MT.inputFifos[0]))

In [31]:
#print(len(MT.partialMatrices))
for x in MT.partialMatrices:
    print(len(x))

3926
10000


In [22]:
#print(MT.partialMatrices)

In [23]:
MT.out

[]

In [33]:
data = []
i = []
j = []
for x in MT.partialMatrices[-1]:
    data.append(x[0])
    i.append(x[1])
    j.append(x[2])

total = coo_matrix((data,(i,j)),shape=(100,100))
print(total.toarray())
print(np.matmul(i1.toarray(),i2.toarray()))
print(np.equal(total.toarray(), np.matmul(i1.toarray(),i2.toarray())))
print(np.allclose(total.toarray(),np.matmul(i1.toarray(),i2.toarray()),rtol=0.01))


[[186058 142576 199097 ... 223029 241214 213323]
 [224050 158783 214551 ... 195794 218845 199443]
 [192675 139708 234021 ... 350837 248405 191099]
 ...
 [274571 179134 285855 ... 292573 244195 216588]
 [319349 185347 298123 ... 314412 259935 261513]
 [240221 172354 289426 ... 183619 252485 191282]]
[[186058 142576 199097 ... 223029 241214 213323]
 [224050 158783 214551 ... 195794 218845 199443]
 [192675 139708 234021 ... 350837 248405 191099]
 ...
 [274571 179134 285855 ... 292573 244195 216588]
 [319349 185347 298123 ... 314412 259935 261513]
 [240221 172354 289426 ... 183619 252485 191282]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
True


In [25]:
print(total.toarray())

[[    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 ...
 [17881  8582 27874 ... 12833 17530  7776]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


In [26]:
print(np.matmul(i1.toarray(),i2.toarray()))

[[186058 142576 199097 ... 223029 241214 213323]
 [224050 158783 214551 ... 195794 218845 199443]
 [192675 139708 234021 ... 350837 248405 191099]
 ...
 [274571 179134 285855 ... 292573 244195 216588]
 [319349 185347 298123 ... 314412 259935 261513]
 [240221 172354 289426 ... 183619 252485 191282]]
